# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
df = pd.read_csv('AviationData.csv', encoding='latin1', low_memory=False)
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [3]:
df_usacode = pd.read_csv('USState_Codes.csv')
df_usacode.head()

,US_State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [4]:
df.shape

(88889, 31)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [6]:
df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

### Cleaning Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

## Amateur Built

In [7]:
sorted(df['Amateur.Built'].dropna().unique())

['No', 'Yes']

In [8]:
df['Amateur.Built'].value_counts()

Amateur.Built
No     80312
Yes     8475
Name: count, dtype: int64

In [9]:
df['Amateur.Built'].isna().sum()

102

In [10]:
# Blank removed and confirmed
df = df.dropna(subset=['Amateur.Built'])
df['Amateur.Built'].isna().sum()

0

In [11]:
# Less than 10% are amateur, eliminate them
df['Amateur.Built'].value_counts(normalize=True)*100

Amateur.Built
No     90.454684
Yes     9.545316
Name: proportion, dtype: float64

In [12]:
df = df[df['Amateur.Built'] != 'Yes']
df['Amateur.Built'].value_counts()

Amateur.Built
No    80312
Name: count, dtype: int64

## EVENT DATE

In [13]:
df['Event.Date']=pd.to_datetime(df['Event.Date'], errors='coerce')

In [14]:
df['Event.Date'].isna().sum()

0

In [15]:
df['Event.Date'].dtype

dtype('<M8[ns]')

In [16]:
df['Event.Date'].dt.year.dropna().unique()

array([1948, 1962, 1974, 1977, 1979, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022])

In [17]:
df['Year'] = df['Event.Date'].dt.year
df['Year'].dropna().unique()

array([1948, 1962, 1974, 1977, 1979, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022])

In [18]:
# Eliminate year < 1983
df = df[df['Year'] >= 1983]
df['Event.Date'].dt.year.dropna().unique()

array([1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76960 entries, 3600 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                76960 non-null  object        
 1   Investigation.Type      76960 non-null  object        
 2   Accident.Number         76960 non-null  object        
 3   Event.Date              76960 non-null  datetime64[ns]
 4   Location                76913 non-null  object        
 5   Country                 76750 non-null  object        
 6   Latitude                30167 non-null  object        
 7   Longitude               30161 non-null  object        
 8   Airport.Code            43375 non-null  object        
 9   Airport.Name            45285 non-null  object        
 10  Injury.Severity         75961 non-null  object        
 11  Aircraft.damage         73868 non-null  object        
 12  Aircraft.Category       25405 non-null  object  

## MAKE AND MODEL

In [20]:
df['Make'].isna().sum()

46

In [21]:
df['Model'].isna().sum()

67

In [22]:
# Eliminate blanks in Make and Models
df = df.dropna(subset=['Make'])
df = df.dropna(subset=['Model'])
df['Make'].isna().sum()

0

In [23]:
df['Model'].isna().sum()

0

In [24]:
df['Make'].value_counts()

Make
Cessna                           20817
Piper                            11260
CESSNA                            4919
Beech                             4052
PIPER                             2838
                                 ...  
AIR TRACTOR INC.                     1
STORCH AVIATION AUSTRALIA PTY        1
Rotorcraft Development Corp          1
Polaris Motors                       1
ROYSE RALPH L                        1
Name: count, Length: 2309, dtype: int64

In [25]:
df['Model'].value_counts()

Model
152          2230
172          1652
172N         1094
PA-28-140     863
172M          759
             ... 
F-21            1
S-T1            1
175-235         1
DHC-7           1
GLASAIR         1
Name: count, Length: 8587, dtype: int64

## Fatal and Serious Injuries

In [26]:
df[df['Total.Fatal.Injuries'] < 0]['Total.Fatal.Injuries'].value_counts()

Series([], Name: count, dtype: int64)

In [27]:
df[df['Total.Serious.Injuries'] < 0]['Total.Serious.Injuries'].value_counts()

Series([], Name: count, dtype: int64)

## Accidents vs Incidents: Investigation.Type

In [28]:
df['Investigation.Type'].value_counts()

Investigation.Type
Accident    73247
Incident     3641
Name: count, dtype: int64

In [29]:
# There is not blakns
df['Investigation.Type'].isna().sum()

0

In [30]:
# the client is only interested in Accidents
df = df[df['Investigation.Type'] == 'Accident']
df['Investigation.Type'].value_counts()

Investigation.Type
Accident    73247
Name: count, dtype: int64

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73247 entries, 3600 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                73247 non-null  object        
 1   Investigation.Type      73247 non-null  object        
 2   Accident.Number         73247 non-null  object        
 3   Event.Date              73247 non-null  datetime64[ns]
 4   Location                73209 non-null  object        
 5   Country                 73057 non-null  object        
 6   Latitude                29184 non-null  object        
 7   Longitude               29178 non-null  object        
 8   Airport.Code            41520 non-null  object        
 9   Airport.Name            43358 non-null  object        
 10  Injury.Severity         72924 non-null  object        
 11  Aircraft.damage         71833 non-null  object        
 12  Aircraft.Category       23786 non-null  object  

In [32]:
df['Injury.Severity'].value_counts()

Injury.Severity
Non-Fatal     57669
Fatal(1)       4693
Fatal          4547
Fatal(2)       3130
Fatal(3)       1046
              ...  
Fatal(144)        1
Fatal(60)         1
Fatal(270)        1
Fatal(143)        1
Fatal(230)        1
Name: count, Length: 103, dtype: int64

## Estimaton of Total Passengers

In [33]:
tot_pas = [
    'Total.Fatal.Injuries',
    'Total.Serious.Injuries',
    'Total.Minor.Injuries',
    'Total.Uninjured'
]

In [34]:
df[tot_pas].isna().all(axis=1).sum()

70

In [35]:
# Change blanks to cero
df[tot_pas] = df[tot_pas].fillna(0)
df[tot_pas].isna().all(axis=1).sum()

0

In [36]:
df['TOTAL_Passengers'] = df['Total.Fatal.Injuries']+ df['Total.Serious.Injuries']+ df['Total.Minor.Injuries']+ df['Total.Uninjured']
df['TOTAL_Passengers'].isna().sum()

0

#### Estimaton of Bad Injuries

In [37]:
df['TOTAL_bad_Injuries'] = df['Total.Fatal.Injuries']+ df['Total.Serious.Injuries']
df['TOTAL_bad_Injuries'].sum()

63439.0

## Injury.Severity

Inspection of 'Injury. Severity' = Unavailable

In [38]:
# There is not a single passenger in 'Unavailable'
df[df['Injury.Severity'] == 'Unavailable']['TOTAL_Passengers'].sum()

0.0

In [39]:
# I will drop all rows with Unavailable because they don't have info of passengers (79)
df[df['Injury.Severity'] == 'Unavailable']['TOTAL_Passengers'].shape

(79,)

In [40]:
# There are 69 blanks /79, the other 10 shows 0 (previous code line) - This step is no needed but it is cool
df[df['Injury.Severity'] == 'Unavailable']['TOTAL_Passengers'].isna().sum()

0

In [41]:
# Delete Unavailable
df = df[df['Injury.Severity'] != 'Unavailable']
# Confirm 
df['Injury.Severity'].dropna().unique()

array(['Non-Fatal', 'Fatal(2)', 'Fatal(3)', 'Fatal(1)', 'Fatal(5)',
       'Fatal(6)', 'Fatal(4)', 'Fatal(23)', 'Fatal(8)', 'Fatal(10)',
       'Fatal(11)', 'Fatal(7)', 'Fatal(9)', 'Fatal(17)', 'Fatal(13)',
       'Fatal(29)', 'Fatal(70)', 'Fatal(135)', 'Fatal(31)', 'Fatal(14)',
       'Fatal(256)', 'Fatal(25)', 'Fatal(82)', 'Fatal(156)', 'Fatal(28)',
       'Fatal(18)', 'Fatal(43)', 'Fatal(15)', 'Fatal(12)', 'Fatal(270)',
       'Fatal(144)', 'Fatal(174)', 'Fatal(111)', 'Fatal(131)',
       'Fatal(20)', 'Fatal(73)', 'Fatal(27)', 'Fatal(34)', 'Fatal(87)',
       'Fatal(30)', 'Fatal(16)', 'Fatal(47)', 'Fatal(56)', 'Fatal(37)',
       'Fatal(132)', 'Fatal(68)', 'Fatal(54)', 'Fatal(52)', 'Fatal(65)',
       'Fatal(72)', 'Fatal(160)', 'Fatal(189)', 'Fatal(123)', 'Fatal(33)',
       'Fatal(110)', 'Fatal(230)', 'Fatal(97)', 'Fatal(349)',
       'Fatal(125)', 'Fatal(228)', 'Fatal(75)', 'Fatal(104)',
       'Fatal(229)', 'Fatal(80)', 'Fatal(217)', 'Fatal(169)', 'Fatal(88)',
       'Fatal(19)',

Inspection of 'Injury. Severity' = blanks

In [42]:
df['Injury.Severity'].isna().sum()

323

In [43]:
# There are 323 blanks and all of them have 0 injuries
df[df['Injury.Severity'].isna()]['TOTAL_Passengers'].sum()

0.0

In [44]:
# Eliminate blanks
df = df.dropna(subset=['Injury.Severity'])
# Confirm
df['Injury.Severity'].isna().sum()

0

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72845 entries, 3600 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                72845 non-null  object        
 1   Investigation.Type      72845 non-null  object        
 2   Accident.Number         72845 non-null  object        
 3   Event.Date              72845 non-null  datetime64[ns]
 4   Location                72810 non-null  object        
 5   Country                 72656 non-null  object        
 6   Latitude                29072 non-null  object        
 7   Longitude               29066 non-null  object        
 8   Airport.Code            41459 non-null  object        
 9   Airport.Name            43302 non-null  object        
 10  Injury.Severity         72845 non-null  object        
 11  Aircraft.damage         71494 non-null  object        
 12  Aircraft.Category       23487 non-null  object  

## Aircraft Damage
The client is only interested in Total destrucction, but I will inspect substancial in injuries

In [46]:
# There are 79 unknows and 647 minor in Aircraft Damage
df['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    55402
Destroyed      15366
Minor            647
Unknown           79
Name: count, dtype: int64

In [47]:
# There are 1351 blanks in Aircraft Damage
df['Aircraft.damage'].isna().sum()

1351

In [48]:
# All 1351 blanks as Unknown
df['Aircraft.damage'] = df['Aircraft.damage'].fillna('Unknown')
df['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    55402
Destroyed      15366
Unknown         1430
Minor            647
Name: count, dtype: int64

In [49]:
df[df['Aircraft.damage'] == 'Unknown']['TOTAL_Passengers'].sum()

78140.0

In [50]:
df[df['Aircraft.damage'] == 'Unknown']['TOTAL_bad_Injuries'].sum()

2409.0

Eliminate 'Unknown' with mild injuries, becasue the bad injuries might be relevants for injuries analisys 

In [51]:
# Delete 
df = df[~((df['Aircraft.damage'] == 'Unknown') & (df['TOTAL_bad_Injuries'].fillna(0) == 0))]
# Confirm 
((df['Aircraft.damage'] == 'Unknown') & (df['TOTAL_bad_Injuries'].fillna(0) == 0)).sum()

0

## Aircraft Category
Only Airplanes meet the conditions of the client

In [52]:
df['Aircraft.Category'].value_counts()

Aircraft.Category
Airplane             19408
Helicopter            2855
Glider                 430
Balloon                139
Weight-Shift           138
Powered Parachute       82
Gyrocraft               31
Ultralight              18
WSFT                     9
Unknown                  7
Blimp                    4
Powered-Lift             3
Rocket                   1
Name: count, dtype: int64

In [53]:
# Delete
df = df[df['Aircraft.Category'] == 'Airplane']
# Confirm
df['Aircraft.Category'].value_counts()

Aircraft.Category
Airplane    19408
Name: count, dtype: int64

## FAR Description

In [54]:
df['FAR.Description'].value_counts()

FAR.Description
091                              13201
Part 91: General Aviation         2500
NUSN                               935
137                                736
135                                489
NUSC                               268
121                                247
Part 137: Agricultural             151
UNK                                145
PUBU                               122
Part 121: Air Carrier               85
Part 135: Air Taxi & Commuter       77
Non-U.S., Non-Commercial            68
129                                 44
Non-U.S., Commercial                39
Part 129: Foreign                   36
Public Use                          11
Unknown                              8
091K                                 7
125                                  3
Part 125: 20+ Pax,6000+ lbs          1
Part 91 Subpart K: Fractional        1
Public Aircraft                      1
ARMF                                 1
107                                  1
Name: cou

In [55]:
df['FAR.Description'].isna().sum()

231

In [56]:
# All 231 blanks as Unknown
df['FAR.Description'] = df['FAR.Description'].fillna('Unknown')
df['FAR.Description'].value_counts()

FAR.Description
091                              13201
Part 91: General Aviation         2500
NUSN                               935
137                                736
135                                489
NUSC                               268
121                                247
Unknown                            239
Part 137: Agricultural             151
UNK                                145
PUBU                               122
Part 121: Air Carrier               85
Part 135: Air Taxi & Commuter       77
Non-U.S., Non-Commercial            68
129                                 44
Non-U.S., Commercial                39
Part 129: Foreign                   36
Public Use                          11
091K                                 7
125                                  3
Part 125: 20+ Pax,6000+ lbs          1
Part 91 Subpart K: Fractional        1
Public Aircraft                      1
ARMF                                 1
107                                  1
Name: cou

In [57]:
# Eliminate the following FAR Descriptions because they are not relevant for the client
# I will keep NUSN and Non-us becasuse it is not clear if they are particular/private
# 137 are Agricultural, unknowns are noise, ARMF military, 107 drones
drop_fars = [
    '137',
    'Part 137: Agricultural',
    'UNK',
    'Unknown',
    'ARMF',
    '107'
]

df['FAR.Description'].isin(drop_fars).sum()

1273

In [58]:
# Delete rows where FAR.Description is in drop_fars
df = df[~df['FAR.Description'].isin(drop_fars)]
# Confirm remaining FAR.Description values
df['FAR.Description'].value_counts()

FAR.Description
091                              13201
Part 91: General Aviation         2500
NUSN                               935
135                                489
NUSC                               268
121                                247
PUBU                               122
Part 121: Air Carrier               85
Part 135: Air Taxi & Commuter       77
Non-U.S., Non-Commercial            68
129                                 44
Non-U.S., Commercial                39
Part 129: Foreign                   36
Public Use                          11
091K                                 7
125                                  3
Part 91 Subpart K: Fractional        1
Public Aircraft                      1
Part 125: 20+ Pax,6000+ lbs          1
Name: count, dtype: int64

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18135 entries, 4171 to 88886
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                18135 non-null  object        
 1   Investigation.Type      18135 non-null  object        
 2   Accident.Number         18135 non-null  object        
 3   Event.Date              18135 non-null  datetime64[ns]
 4   Location                18133 non-null  object        
 5   Country                 18134 non-null  object        
 6   Latitude                17222 non-null  object        
 7   Longitude               17219 non-null  object        
 8   Airport.Code            12797 non-null  object        
 9   Airport.Name            12814 non-null  object        
 10  Injury.Severity         18135 non-null  object        
 11  Aircraft.damage         18135 non-null  object        
 12  Aircraft.Category       18135 non-null  object  

### Constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

## RATE BAD INJURIES / Number of Passengers

In [60]:
# I don't want to eliminate 
(df['TOTAL_Passengers'] == 0).sum()

18

In [61]:
# I don't want to eliminate 
(df['TOTAL_bad_Injuries'] == 0).sum()

11947

In [62]:
# Following data has 0 passengers but high damage to the aircraft.
# I will exclude them for the rate but still in the data because they could be relevant for damages
df[df['TOTAL_Passengers'] == 0][[
    'Event.Date', 'FAR.Description', 'Aircraft.Category', 'FAR.Description',
    'Aircraft.damage', 'Purpose.of.flight',
    'TOTAL_Passengers'
]]

,Event.Date,FAR.Description,Aircraft.Category,FAR.Description,Aircraft.damage,Purpose.of.flight,TOTAL_Passengers
46835,1999-08-13,Part 91: General Aviation,Airplane,Part 91: General Aviation,Substantial,Unknown,0.0
47320,1999-10-16,Part 91: General Aviation,Airplane,Part 91: General Aviation,Substantial,Unknown,0.0
73537,2013-05-10,PUBU,Airplane,PUBU,Substantial,Public Aircraft - Federal,0.0
73963,2013-07-27,PUBU,Airplane,PUBU,Substantial,Aerial Observation,0.0
74696,2014-01-27,PUBU,Airplane,PUBU,Destroyed,Public Aircraft - Federal,0.0
76571,2015-05-01,091,Airplane,091,Destroyed,Flight Test,0.0
79171,2016-10-25,NUSN,Airplane,NUSN,Substantial,NaN,0.0
84080,2019-10-17,091,Airplane,091,Substantial,Flight Test,0.0
84230,2019-11-25,091,Airplane,091,Substantial,Instructional,0.0
85602,2020-11-13,PUBU,Airplane,PUBU,Destroyed,Public Aircraft - Federal,0.0


In [63]:
# If tehre is not injuries and no passanger, I will take that as cero
df['Bad_Injuries_Rate'] = df['TOTAL_bad_Injuries'] / df['TOTAL_Passengers'].replace(0, np.nan)
df['Bad_Injuries_Rate'].head(20)

4171     1.000000
4285     0.200000
5960     0.500000
6669     0.033333
6806     0.000000
7084     0.000000
7708     0.000000
8585     0.000000
8591     0.000000
10247    0.000000
10605    1.000000
10688    0.500000
11638    1.000000
11898    0.333333
12384    0.000000
13114    1.000000
14259    1.000000
14357    0.000000
14711    0.666667
14712    0.666667
Name: Bad_Injuries_Rate, dtype: float64

## Indicators for Fatal/Serious Injuries

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

## Engine Type

no risk, 1 unknow clean

In [64]:
df['Engine.Type'].value_counts()

Engine.Type
Reciprocating    14368
Turbo Prop         720
Turbo Fan          450
Turbo Jet           92
Unknown             67
Turbo Shaft          5
Electric             4
UNK                  1
Name: count, dtype: int64

In [65]:
df = df[df['Engine.Type'] != 'UNK']
df['Engine.Type'].value_counts()

Engine.Type
Reciprocating    14368
Turbo Prop         720
Turbo Fan          450
Turbo Jet           92
Unknown             67
Turbo Shaft          5
Electric             4
Name: count, dtype: int64

## Weather Condition

small risk, 171 unknown , clean

In [66]:
df['Weather.Condition'].value_counts()

Weather.Condition
VMC    15670
IMC      987
Unk      129
UNK       42
Name: count, dtype: int64

In [67]:
df = df[df['Weather.Condition'] != 'UNK']
df = df[df['Weather.Condition'] != 'Unk']
df['Weather.Condition'].value_counts()

Weather.Condition
VMC    15670
IMC      987
Name: count, dtype: int64

## Num of Engines

NO risk

In [68]:
df['Number.of.Engines'].value_counts()

Number.of.Engines
1.0    14522
2.0     2176
4.0       35
3.0       22
0.0        4
8.0        1
6.0        1
Name: count, dtype: int64

## Broad.phase.of.flight

10 unknown , low risk and no relevant for the evaluation

In [69]:
df['Broad.phase.of.flight'].value_counts()

Broad.phase.of.flight
Landing        1227
Takeoff         449
Cruise          245
Approach        230
Taxi            104
Maneuvering     101
Go-around        89
Descent          66
Climb            51
Standing         41
Unknown           8
Other             3
Name: count, dtype: int64

## Porpouse of the flight

NO risk

In [70]:
df['Purpose.of.flight'].value_counts()

Purpose.of.flight
Personal                     11533
Instructional                 2694
Business                       464
Positioning                    321
Unknown                        237
Skydiving                      162
Aerial Observation             152
Other Work Use                 150
Flight Test                    114
Ferry                           98
Banner Tow                      89
Executive/corporate             89
Aerial Application              59
Public Aircraft - Federal       50
Air Race show                   47
Glider Tow                      35
Public Aircraft                 35
Public Aircraft - State         23
Firefighting                    15
Public Aircraft - Local         11
ASHO                             5
Air Race/show                    4
Air Drop                         3
PUBS                             3
Name: count, dtype: int64

## Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17963 entries, 4171 to 88886
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                17963 non-null  object        
 1   Investigation.Type      17963 non-null  object        
 2   Accident.Number         17963 non-null  object        
 3   Event.Date              17963 non-null  datetime64[ns]
 4   Location                17963 non-null  object        
 5   Country                 17962 non-null  object        
 6   Latitude                17110 non-null  object        
 7   Longitude               17107 non-null  object        
 8   Airport.Code            12756 non-null  object        
 9   Airport.Name            12770 non-null  object        
 10  Injury.Severity         17963 non-null  object        
 11  Aircraft.damage         17963 non-null  object        
 12  Aircraft.Category       17963 non-null  object  

In [72]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date', 'Year', 'TOTAL_Passengers', 'TOTAL_bad_Injuries',
       'Bad_Injuries_Rate'],
      dtype='object')

In [73]:
df.to_csv("AviationData_cleaned.csv", index=False)

# DONE

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

## DONE

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

## DONE

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

## DONE

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs